# Preparation
net 6 SDK installieren

In [ ]:
dotnet tool install --global dotnet-ef                              


# neues Dart project

In [ ]:
mkdir darts
cd darts
git init
dotnet new gitignore
dotnet new console -o .\EFmanymanymany
dotnet new sln
dotnet sln add .\EFmanymanymany
cd .\EFmanymanymany
dotnet add package Microsoft.EntityFrameworkCore.Design 
dotnet add package Microsoft.EntityFrameworkCore.SqlServer
dotnet ef dbcontext scaffold --context DartsContext "Data Source=(localdb)\MSSQLLocalDB;Initial Catalog=darts;Integrated Security=True;Connect Timeout=30;Encrypt=False;TrustServerCertificate=False;ApplicationIntent=ReadWrite;MultiSubnetFailover=False" Microsoft.EntityFrameworkCore.SqlServer
git add *
git commit -m "initial"

add some models
dont forget to add them as DBSet in DartsContext

In [ ]:
dotnet ef migrations add Initial
dotnet ef database update